In [1]:
from lxml import etree

In [2]:

path_in = r"C:\Users\yagam\Downloads\1373_ChroniqueNormande_convTags_ambi_17.03.23.xml"                  

path_out = r"C:\Users\yagam\Downloads\1373_ChroniqueNormande_convTags_ambi_17.03.23.xml"

def check_coherence(l, prpos, uppos, udpos):
    
    global incoherence
    
    if udpos == 'PUNCT' or udpos == 'PROPN':
        pass
    elif (prpos, uppos, udpos) not in l:
        if udpos == 'VERB':
            if (prpos, uppos, 'AUX') not in l:
                incoherence = True
        elif udpos == 'AUX':
            if (prpos, uppos, 'VERB') not in l:
                incoherence = True
        else:
            incoherence = True
    
    return incoherence

CorrTable = open(path_corrTable, encoding='utf-8')
l = []
for i in CorrTable:
    i = i.split(",")
    l.append((i[2], i[3], i[4]))

l.append(('','_', '_'))
l.append((None,None,None))

CorrTable.close()

In [3]:
tree = etree.parse(open(path_in, encoding='utf-8'))
tree_tokenized = tree.getroot()

test_ = []
l_numSG = []

for id_, s in enumerate(tree_tokenized.findall('.//s')):
    
    for w in s.findall('.//w'):
        
        if w.get('uppos') ==  'NCS':
            id_w = w.get('n')
            
            for w2 in tree_tokenized.findall('.//s')[id_]:
                
                if w2.get('head') == id_w and w2.get('function') == 'nummod':
                    
                    if w.text in l_numSG:
                        pass
                    else:
                        w.set('uppos', 'NCPL')
                        w.set('annot_NCPL', "nummod")
                
                if w2.get('head') == id_w and (w2.text.endswith('s') or w2.text.endswith('z') or w2.text.endswith('x')) and w2.get('udpos')=='DET':
                    w.set('uppos', 'NCPL')
                    w.set('annot_NCPL', 'DET_s')
                                    
                            

In [4]:
def indent(elem, level=0, more_sibs=False):
    i = "\n"
    if level:
        i += (level-1) * '\t'
    num_kids = len(elem)
    if num_kids:
        if not elem.text or not elem.text.strip():
            elem.text = i + "\t"
            if level:
                elem.text += '\t'
        count = 0
        for kid in elem:
            indent(kid, level+1, count < num_kids - 1)
            count += 1
        if not elem.tail or not elem.tail.strip():
            elem.tail = i
            if more_sibs:
                elem.tail += '\t'
    else:
        if level and (not elem.tail or not elem.tail.strip()):
            elem.tail = i
            if more_sibs:
                elem.tail += '\t'

In [5]:
indent(tree_tokenized)
#etree.dump(new_root)
etree.ElementTree(tree_tokenized).write(path_out, encoding='utf-8')